In [1]:
import pandas as pd
import numpy as np
import os
import glob as gb
import sys
sys.path.insert(0, "../../")
from clases.bd.conexion2 import MyDatabase2
conn = MyDatabase2()


ModuleNotFoundError: No module named 'psycopg2'

In [2]:


ruta_datos = os.path.abspath("../../../data/2023/csv/02-2023.csv")
csv=pd.read_csv(ruta_datos, encoding="ISO-8859-1")
csv=csv[csv['Peso']>999]

result = csv.groupby('Peso').size().reset_index(name='count')
print(result)


FileNotFoundError: [Errno 2] No such file or directory: 'd:\\Irvin\\Irvin\\Python\\data\\2023\\csv\\02-2023.csv'

In [ ]:
def consulta_C0010(conn):  
    sql = conn.df("""SELECT *
FROM( 
   SELECT nt.fecha_atencion, nt.id_paciente , nt.id_cita, mp2.id_tipo_documento ,mp2.numero_documento , nt.fecha_atencion,nt.codigo_item,mhcc.descripcion_item,nt.id_ups,mhu.descripcion_ups,concat(mr.nombres_registrador,' ',mr.apellido_paterno_registrador,' ',mr.apellido_materno_registrador )AS registrador ,concat(mp.nombres_personal,' ',mp.apellido_paterno_personal,' ',mp.apellido_materno_personal) AS personal, nt.id_otra_condicion , 
 max(CASE  WHEN nt.id_correlativo_lab='1' THEN nt.valor_lab ELSE null END ) AS Lab1,
 max(CASE  WHEN nt.id_correlativo_lab='2' THEN nt.valor_lab ELSE null END ) AS Lab2,
 max(CASE  WHEN nt.id_correlativo_lab='3' THEN nt.valor_lab ELSE null END ) AS Lab3
FROM maestros.nominal_trama nt
INNER JOIN maestros.maestro_paciente mp2 ON nt.id_paciente=mp2.id_paciente 
left JOIN maestros.maestro_his_cie_cpms mhcc ON nt.codigo_item =mhcc.codigo_item 
LEFT JOIN maestros.maestro_his_ups mhu ON mhu.id_ups = nt.id_ups 
LEFT JOIN maestros.maestro_registrador mr ON mr.id_registrador =nt.id_registrador 
LEFT JOIN maestros.maestro_personal mp ON mp.id_personal =nt.id_personal 
WHERE nt.codigo_item ='C0010' AND nt.fecha_atencion BETWEEN '2023-06-01' AND '2023-07-31'  AND mp2.id_tipo_documento =1 AND nt.id_otra_condicion =1
GROUP BY  nt.id_paciente , nt.id_cita, mp2.id_tipo_documento ,mp2.numero_documento, nt.fecha_atencion,nt.codigo_item,mhcc.descripcion_item,nt.id_ups,mhu.descripcion_ups,mr.nombres_registrador,mr.apellido_paterno_registrador,mr.apellido_materno_registrador , mp.nombres_personal,mp.apellido_paterno_personal,mp.apellido_materno_personal,nt.id_otra_condicion
) AS t WHERE Lab1='ALI' AND Lab2='GL'   """)
    
    sql = sql.rename(columns={'numero_documento': 'dni'})
    sql['dni'] = sql['dni'].astype(int)
    
    
    return sql
df_dx=consulta_C0010(conn)

In [ ]:
def consulta_sql(conn):  

    sql = conn.df(""" SELECT * FROM nominal_trama nt WHERE nt.fecha_atencion  BETWEEN '2023-08-01' AND '2023-08-15'  """)
    return sql
df_1=consulta_sql(conn)

In [ ]:
df=df_1
# Eliminar duplicados
df_no_duplicates = df.drop_duplicates(subset=['id_paciente', 'codigo_item','fecha_atencion','id_correlativo_lab'])

# Pivoteo dinámico usando pandas
pivoted_df = df_no_duplicates.pivot(index=['id_cita','id_paciente','fecha_atencion','codigo_item', 'tipo_diagnostico'],
                                     columns='id_correlativo_lab',
                                     values='valor_lab').reset_index()
print(pivoted_df)
pivoted_df.to_excel('demo.xlsx', index=False)

In [ ]:
def consulta_sql(conn):  

    sql = conn.df(""" SELECT * FROM maestros.maestro_his_ubigeo_inei_reniec """)
    sql = sql.rename(columns={'id_ubigueo_inei': 'ubigeo'})
    return sql
ubigeo=consulta_sql(conn)

<h1>NORKA - NIÑO</H1>

In [29]:
#archivo_excel = "nominal_ninos.xlsx"
doc="SELLO ESTADISTICA/Niños agos a nov 6 a 11 meses"
archivo_excel = """%s.xlsx"""%doc

df = pd.read_excel(archivo_excel,sheet_name=0)
#df['ubigeo'] = df['ubigeo'].astype(str).str.zfill(6)

df=df.drop_duplicates(subset=['Nro_Doc'])

df = df.rename(columns={'Nro_Doc': 'nro_doc'})

##############################################################################################

# Convierte los DNIs a una lista
dnis = df['nro_doc'].tolist()
# Convierte los DNIs en una cadena separada por comas para usar en la consulta SQL
numeros_dni_str = ','.join(["'" + str(dni) + "'" for dni in dnis])

# Crea la consulta SQL con una cláusula WHERE para filtrar por los DNIs
# Consulta SQL con los números de DNI
sql = f"""SELECT t.numero_documento as nro_doc,t.rn as his  FROM (
SELECT
   nt.numero_documento, 
    ROW_NUMBER() OVER (PARTITION BY nt.numero_documento ORDER BY nt.fecha_atencion ) AS rn
FROM
    maestros.nominal_trama2 nt
WHERE
    nt.codigo_item = 'C0010'
    AND nt.lab1 = 'ALI'
    AND nt.lab2 = 'GL'
    AND nt.mes in(8,9,10,11,12)
    AND nt.numero_documento IN ({numeros_dni_str}) )t WHERE t.rn=1 """ 
      
ninio=conn.df(sql)

#######################################################################################################

ninio['nro_doc'] = ninio['nro_doc'].astype(int)

df=df.merge(ninio, how='left', on=['nro_doc'])

#####################################################################################################

#archivo_excel = "nominal_ninos.xlsx"
doc="SELLO ESTADISTICA/NOMINA DE NIÑOS Y GESTANTES CONSOLIDADO 14.12.23"
archivo_excel = """%s.xlsx"""%doc

df2 = pd.read_excel(archivo_excel,sheet_name=0)
df2=df2.drop_duplicates(subset=['N° DNI'])

df2 = df2.rename(columns={'N° DNI': 'nro_doc'})
df2 = df2.drop(['PROVINCIA', 'DISTRITO','Nombre y Apellidos','FECHA DE NACIMIENTO','EDAD'	], axis=1)
df2['numerador'] = 1

df=df.merge(df2, how='left', on=['nro_doc'])

df.to_excel("SELLO ESTADISTICA/ESTADISTICA/Niños agos a nov 6 a 11 meses_Final.xlsx", index=False)


d:\Irvin\Irvin\Python\Python-GERESA\modulos\pruebas\../..\clases\bd\conexion2.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


<h1>NORKA - GESTANTE</H1>

In [30]:
#archivo_excel = "nominal_ninos.xlsx"
doc="SELLO ESTADISTICA/nomina gestante agost a noviembre"
archivo_excel = """%s.xlsx"""%doc

df = pd.read_excel(archivo_excel,sheet_name=0)
#df['ubigeo'] = df['ubigeo'].astype(str).str.zfill(6)

df=df.drop_duplicates(subset=['Nro_Doc'])
df = df.rename(columns={'Nro_Doc': 'nro_doc'})
##############################################################################################

# Convierte los DNIs a una lista
dnis = df['nro_doc'].tolist()
# Convierte los DNIs en una cadena separada por comas para usar en la consulta SQL
numeros_dni_str = ','.join(["'" + str(dni) + "'" for dni in dnis])

# Crea la consulta SQL con una cláusula WHERE para filtrar por los DNIs
# Consulta SQL con los números de DNI
sql = f"""SELECT t.numero_documento as nro_doc,t.rn as his FROM (
SELECT
   nt.numero_documento, nt.codigo_item , nt.id_otra_condicion, nt.lab1 , nt.lab2 ,
    ROW_NUMBER() OVER (PARTITION BY nt.numero_documento ORDER BY nt.fecha_atencion ) AS rn
FROM
    maestros.nominal_trama2 nt  
WHERE
    nt.codigo_item = 'C0010'
 AND nt.lab1 = 'ALI'
    AND nt.lab2 = 'GL'
    AND nt.mes in(8,9,10,11,12)
    AND nt.anio =2023
    AND nt.id_otra_condicion =1
    AND nt.numero_documento in ({numeros_dni_str})   
    ) t WHERE t.rn=1 """       

gestante=conn.df(sql)

gestante['nro_doc'] = gestante['nro_doc'].astype(int)

df=df.merge(gestante, how='left', on=['nro_doc'])

#######################################################################################################

#archivo_excel = "nominal_ninos.xlsx"
doc="SELLO ESTADISTICA/NOMINA DE NIÑOS Y GESTANTES CONSOLIDADO 14.12.23"
archivo_excel = """%s.xlsx"""%doc

df2 = pd.read_excel(archivo_excel,sheet_name=1)
df2 = df2.rename(columns={'N° DNI': 'nro_doc'})
df2=df2.drop_duplicates(subset=['nro_doc'])

columnas_a_conservar = ['nro_doc', ]
# Eliminar todas las columnas excepto las especificadas en columnas_a_conservar
df2.drop(df2.columns.difference(columnas_a_conservar), axis=1, inplace=True)

df2['numerador'] = 1

df=df.merge(df2, how='left', on=['nro_doc'])

df.to_excel("SELLO ESTADISTICA/ESTADISTICA/nomina gestante agost a noviembre.xlsx", index=False)

d:\Irvin\Irvin\Python\Python-GERESA\modulos\pruebas\../..\clases\bd\conexion2.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


In [ ]:


df = pd.read_excel(archivo_excel,sheet_name=0)

In [ ]:
import paramiko

# Configuramos los datos de conexión
#host = "sftp://carga.minsa.gob.pe"
host = "181.177.250.128"
port = 22
username = "usr_dhr_008"
password = "X3veTg8z62R74J"

# Creamos el objeto SSHClient
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(host, port, username, password)

# Creamos el objeto SFTPClient
sftp = client.open_sftp()

# Descargamos el archivo
filename = "/data_cvs_nominal/2023/"
local_path = "C:/Users/IRVIN/Downloads/his/"
sftp.get(filename, local_path)

# Cerramos la conexión
sftp.close()
client.close()

In [ ]:
import paramiko
import os

def download_files_sftp(hostname, port, username, password, remote_path, local_path):
    try:
        # Crear el objeto SSHClient
        client = paramiko.SSHClient()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

        # Conectar al servidor SFTP
        client.connect(hostname, port, username, password)

        # Crear el objeto SFTPClient
        sftp = client.open_sftp()

        # Verificar si la ruta local existe, si no, crearla
        if not os.path.exists(local_path):
            os.makedirs(local_path)

        # Listar archivos remotos en el directorio
        remote_files = sftp.listdir(remote_path)

        # Descargar cada archivo remoto
        for remote_file in remote_files:
            remote_file_path = os.path.join(remote_path, remote_file)
            local_file_path = os.path.join(local_path, remote_file)
            sftp.get(remote_file_path, local_file_path)

        print("Descarga completa.")

    except Exception as e:
        print(f"Error durante la descarga: {e}")

    finally:
        # Cerrar la conexión
        if sftp:
            sftp.close()
        client.close()

# Configuramos los datos de conexión
host = "181.177.250.128"
port = 22
username = "usr_dhr_008"
password = "X3veTg8z62R74J"

# Ruta remota y local
remote_directory = "/data_cvs_nominal/2023/"
local_directory = "C:/Users/IRVIN/Downloads/his/"

# Llamar a la función para descargar archivos
download_files_sftp(host, port, username, password, remote_directory, local_directory)


In [ ]:
import pandas as pd

archivo_excel = 'D:\Irvin\Anemia\Capacitacion\Curso Procesamiento Bases His Cusco-Puno-Piura\Materiales de consulta\RENAES - Taller.xlsx'
datos = pd.read_excel(archivo_excel)


In [ ]:
datos.columns = datos.columns.str.lower()
conn.insert_df(datos, 'renaes_altitud_eess','public')

In [1]:
from decouple import config
print (config('DB_HOST'))
print (config('DB_USER'))
print (config('DB_PASSWORD'))
print (config('DB_DATABASE'))
print (config('DB_PORT'))


192.168.1.7
postgres
213141
irvin_hisminsa
5432
